# 航空公司客户价值分析（案例）

This notebook runs on Python 3.5 with Spark 2.1.
___________

## 内容列表

1.  [实现目标](#lrfmc_1)<br>
2.  [数据探索](#lrfmc_2)<br>
3.  [数据处理](#lrfmc_3) <br>
 3.1.  [数据清洗，过滤掉不符合规则的数据](#lrfmc_3_1) <br>
 3.2.  [数据转换](#lrfmc_3_2) <br>
 3.3.  [标准差标准化](#lrfmc_3_3) <br>
4.  [模型构建](#lrfmc_4) <br>
 4.1.  [K-Means聚类算法,对数据进行分类](#lrfmc_4_1) <br>
 4.2.  [聚类结果分析](#lrfmc_4_2) <br>
5.  [模型拟合原数据](#lrfmc_5) <br>
 5.1.  [将客户分类类型拟合到原数据](#lrfmc_5_1) <br>
 5.2.  [对数据进行统计](#lrfmc_5_2) <br>

 <a id="lrfmc_1"></a>
# 1.根据客户数据,实现以下目标:
- 借助航空公司数据,对客户进行分类<br>
- 对不同的客户进行特征分析,比较不同的客户价值<br>
- 对不同客户价值的客户提供不同服务,指定相应的营销策略<br>

In [327]:
import pandas as pd
from sklearn.cluster import KMeans
from io import StringIO
import requests
import json
import numpy as np
import brunel

In [328]:
# The code was removed by DSX for sharing.

,MEMBER_NO,FFP_DATE,FIRST_FLIGHT_DATE,GENDER,FFP_TIER,WORK_CITY,WORK_PROVINCE,WORK_COUNTRY,AGE,LOAD_TIME,...,ADD_Point_SUM,Eli_Add_Point_Sum,L1Y_ELi_Add_Points,Points_Sum,L1Y_Points_Sum,Ration_L1Y_Flight_Count,Ration_P1Y_Flight_Count,Ration_P1Y_BPS,Ration_L1Y_BPS,Point_NotFlight
0,54993,2006/11/02,2008/12/24,男,6,.,北京,CN,31.0,2014/03/31,...,39992,114452,111100,619760,370211,0.509524,0.490476,0.487221,0.512777,50
1,28065,2007/02/19,2007/08/03,男,6,NaN,北京,CN,42.0,2014/03/31,...,12000,53288,53288,415768,238410,0.514286,0.485714,0.489289,0.510708,33
2,55106,2007/02/01,2007/08/30,男,6,.,北京,CN,40.0,2014/03/31,...,15491,55202,51711,406361,233798,0.518519,0.481481,0.481467,0.518530,26
3,21189,2008/08/22,2008/08/23,男,5,Los Angeles,CA,US,64.0,2014/03/31,...,0,34890,34890,372204,186100,0.434783,0.565217,0.551722,0.448275,12
4,39546,2009/04/10,2009/04/15,男,6,贵阳,贵州,CN,48.0,2014/03/31,...,22704,64969,64969,338813,210365,0.532895,0.467105,0.469054,0.530943,39


<a id="lrfmc_2"></a>
# 2.对数据进行基本的探索
### 检查空值以及最小值

In [329]:
data_7_1 = df_data_1.describe(percentiles = [], include = 'all').T #包括对数据的基本描述，percentiles参数是指定计算多少的分位数表（如1/4分位数、中位数等）；T是转置，转置后更方便查阅
data_7_1['null'] = len(df_data_1)-data_7_1['count'] #describe()函数自动计算非空值数，需要手动计算空值数

data_7_1 = data_7_1[['null', 'max', 'min']]
data_7_1.columns = [u'空值数', u'最大值', u'最小值'] #表头重命名
'''这里只选取部分探索结果。
describe()函数自动计算的字段有count（非空值数）、unique（唯一值数）、top（频数最高者）、freq（最高频数）、mean（平均值）、std（方差）、min（最小值）、50%（中位数）、max（最大值）
'''
print(data_7_1.tail(5))

                        空值数       最大值 最小值
Ration_L1Y_Flight_Count   0         1   0
Ration_P1Y_Flight_Count   0         1   0
Ration_P1Y_BPS            0  0.999989   0
Ration_L1Y_BPS            0  0.999993   0
Point_NotFlight           0       140   0


/usr/local/src/conda3_runtime.v21/4.1.1/lib/python3.5/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


<a id="lrfmc_3"></a>
# 3.数据处理
<a id="lrfmc_3_1"></a>
### 3.1 数据清洗，过滤掉不符合规则的数据<br>
丢弃票价为空的记录<br>
丢弃票价为0,平均折扣率为0,总飞行公里数大于0的记录

In [330]:
data_7_2 = df_data_1.copy()
data_7_2 = data_7_2[data_7_2['SUM_YR_1'].notnull() & data_7_2['SUM_YR_2'].notnull()] #票价非空值才保留

#只保留票价非零的，或者平均折扣率与总飞行公里数同时为0的记录。
index1 = data_7_2['SUM_YR_1'] != 0
index2 = data_7_2['SUM_YR_2'] != 0
index3 = (data_7_2['SEG_KM_SUM'] == 0) & (data_7_2['avg_discount'] == 0) #该规则是“与”
data_7_2 = data_7_2[index1 | index2 | index3] #该规则是“或”
print(data_7_2.head(5))

   MEMBER_NO    FFP_DATE FIRST_FLIGHT_DATE GENDER  FFP_TIER    WORK_CITY  \
0      54993  2006/11/02        2008/12/24      男         6            .   
1      28065  2007/02/19        2007/08/03      男         6          NaN   
2      55106  2007/02/01        2007/08/30      男         6            .   
3      21189  2008/08/22        2008/08/23      男         5  Los Angeles   
4      39546  2009/04/10        2009/04/15      男         6           贵阳   

  WORK_PROVINCE WORK_COUNTRY   AGE   LOAD_TIME       ...         \
0            北京           CN  31.0  2014/03/31       ...          
1            北京           CN  42.0  2014/03/31       ...          
2            北京           CN  40.0  2014/03/31       ...          
3            CA           US  64.0  2014/03/31       ...          
4            贵州           CN  48.0  2014/03/31       ...          

   ADD_Point_SUM  Eli_Add_Point_Sum  L1Y_ELi_Add_Points  Points_Sum  \
0          39992             114452              111100      619760  

<a id="lrfmc_3_2"></a>
### 3.2 数据转换
#### LRFMC是验证客户是否是优质客户的指标
-  L = LOAD_TIME - FFP_DATE  -- 会员入会时间距观测窗口结束的月数 = 观测窗口的结束时间 - 入会时间<br>
-  R = LAST_TO_END        -- 最近一次乘坐距观测窗口结束的月数 = 最近一次乘坐时间至观察窗口末端时长<br>
-  F = FLIGHT_COUNT       -- 客户在观察窗口内乘坐次数 = 观测窗口的飞行次数<br>
-  M = SEG_KM_SUM        -- 客户在观察窗口内累计飞行里程 = 观测窗口的总飞行里数<br>
-  C = avg_discount       -- 客户在观察窗口内乘坐舱位折扣系数平均值 = 平均折扣率<br>

In [331]:
data_7_2_rst = pd.DataFrame()
data_7_2_rst['L'] = pd.to_datetime(data_7_2['LOAD_TIME']) - pd.to_datetime(data_7_2['FFP_DATE'])
data_7_2_rst['L'] = data_7_2_rst.L.map(lambda x: x/np.timedelta64(1*60*60*24*30, 's'))
data_7_2_rst['R'] = data_7_2['LAST_TO_END'] / 30
data_7_2_rst['F'] = data_7_2['FLIGHT_COUNT']
data_7_2_rst['M'] = data_7_2['SEG_KM_SUM']
data_7_2_rst['C'] = data_7_2['avg_discount']
data_7_2_rst = data_7_2_rst.round(2)
print (data_7_2_rst.head(5))

       L     R    F       M     C
0  90.20  0.03  210  580717  0.96
1  86.57  0.23  140  293678  1.25
2  87.17  0.37  135  283712  1.25
3  68.23  3.23   23  281336  1.09
4  60.53  0.17  152  309928  0.97


<a id="lrfmc_3_3"></a>
### 3.3 标准差标准化

In [332]:
data_7_3 = (data_7_2_rst - data_7_2_rst.mean(axis = 0))/(data_7_2_rst.std(axis = 0)) #简洁的语句实现了标准化变换，类似地可以实现任何想要的变换。
data_7_3.columns=['Z'+i for i in data_7_3.columns] #表头重命名。
print(data_7_3.head(5))

         ZL        ZR         ZF         ZM        ZC
0  1.435706 -0.945491  14.034016  26.761154  1.286487
1  1.307268 -0.912438   9.073213  13.126864  2.855183
2  1.328498 -0.889301   8.718869  12.653481  2.855183
3  0.658357 -0.416644   0.781585  12.540622  1.989695
4  0.385913 -0.922354   9.923636  13.898736  1.340580


<a id="lrfmc_4"></a>
# 4.模型构建
<a id="lrfmc_4_1"></a>
#### 4.1 K-Means聚类算法,对数据进行分类

In [333]:
k = 5        #需要进行的聚类类别数,这个demo即会生成 k 种类型的客户
#调用k-means算法，进行聚类分析
kmodel = KMeans(n_clusters = k, n_jobs = 1) #n_jobs是并行数，一般等于CPU数较好
kmodel.fit(data_7_3) #训练模型

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=5, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

<a id="lrfmc_4_2"></a>
#### 4.2 聚类结果分析:
- 重要保持客户:<br>
    这类客户平均折扣率(C)较高(乘坐的航班舱位等级高),最近乘坐本公司航班(R)低,次数(F)高或者里程(M)较高,这类客户贡献大,但是比例小,是理想客户类型<br>
- 重要发展客户:<br>
    这类客户平均折扣率(C)较高,最近乘坐本公司航班(R)低,次数(F)高或者里程(M)较低,入会时间(L)短,这类客户当前价值不高,但是有很大发展潜力<br>
- 重要挽留客户:<br>
    这类客户平均折扣率(C),次数(F)高或者里程(M)较高,但长时间没有坐过航班或者乘坐频率变小(R)高,这类客户价值变化不确定,客户衰退原因各不相同,公司应根据客户异常消费列出客户名单重点联系<br>
- 一般与低价值客户:<br>
    这类客户平均折扣率(C)很低,但长时间没有坐过航班(R)高,次数(F)高或者里程(M)较低,入会时间(L)短,这类客户是一般低价值客户,偶尔乘坐或者在打折促销时才会乘坐本公司航班<br>
    
##### 根据分析的客户类型可以看出模型导出的五种客户类型:
- 类型1: 入会时间(L)短,折扣率(C)低,次数(F)里程(M)较高,属于一般客户
- 类型2: 入会时间(L)短,,(R)高,只是最近入会而且偶尔乘坐,属于低价值客户
- 类型3: 折扣率(C)高,入会时间(L)中规中矩,但是次数(F)里程(M)较低,说明刚开始乘坐本公司航班,但有价值,属于重要发展客户
- 类型4: 入会时间(L)次数(F)里程(M)平均折扣率(C)都高,(R)低,说明经常做而且舱位较高,属于重要保持客户
- 类型5: 入会时间(L)较长,同时次数(F)里程(M)略高,属于重要挽留客户

In [334]:
kmodel_type = kmodel.cluster_centers_  #聚类产生的 k 种客户类型(每次返回的数据的顺序可能不同)
kmodel_value = kmodel.labels_           #每条数据对应聚类的结果

df_kmodel_type = pd.DataFrame(kmodel_type)
df_kmodel_type['TYPE'] = [x for x in range(5)] #客户类型标签
df_kmodel_type.columns = ['L','R','F','M','C','TYPE']
df_kmodel_type = df_kmodel_type.sort('L')

print("客户类型种类:",df_kmodel_type)
print("每个客户所对应的客户类型:",kmodel_value)

客户类型种类:           L         R         F         M         C  TYPE
2 -0.697193 -0.408370 -0.167560 -0.168931 -0.211528     2
0 -0.313806  1.670194 -0.573242 -0.537621 -0.133217     0
3  0.170236 -0.079617 -0.102828 -0.084144  2.696742     3
4  0.482482 -0.798478  2.485007  2.425408  0.280130     4
1  1.151354 -0.369719 -0.094744 -0.103717 -0.128328     1
每个客户所对应的客户类型: [4 4 4 ..., 2 0 0]


/gpfs/fs01/user/sf31-bf990ab5b3efba-4276a445e7a2/.local/lib/python3.5/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


<a id="lrfmc_5"></a>
# 5.模型拟合原数据
<a id="lrfmc_5_1"></a>
### 5.1 将客户分类类型拟合到原数据

In [335]:
member_type = []
#客户分类后具体的客户类型
df_kmodel_type['TYPE_NAME'] = ["一般客户","低价值客户","重要发展客户","重要保持客户","重要挽留客户"]
for i in kmodel_value:
    member_type.append(df_kmodel_type.loc[i].TYPE_NAME)
data_7_2.insert(1,'TYPE',member_type)
index_val = data_7_2.index
member_type_data_1 = []
for index, val in df_data_1.iterrows():
    if index in (index_val.values):
        member_type_data_1.append(data_7_2.loc[index].TYPE)
    else:
        #数据无法统计的客户
        member_type_data_1.append("NA")
                                  
df_data_1.insert(1,'TYPE',member_type_data_1)
print(df_data_1.head(10))


   MEMBER_NO    TYPE    FFP_DATE FIRST_FLIGHT_DATE GENDER  FFP_TIER  \
0      54993  重要保持客户  2006/11/02        2008/12/24      男         6   
1      28065  重要保持客户  2007/02/19        2007/08/03      男         6   
2      55106  重要保持客户  2007/02/01        2007/08/30      男         6   
3      21189  重要保持客户  2008/08/22        2008/08/23      男         5   
4      39546  重要保持客户  2009/04/10        2009/04/15      男         6   
5      56972  重要保持客户  2008/02/10        2009/09/29      男         6   
6      44924  重要保持客户  2006/03/22        2006/03/29      男         6   
7      22631  重要保持客户  2010/04/09        2010/04/09      女         6   
8      32197  重要保持客户  2011/06/07        2011/07/01      男         5   
9      31645  重要保持客户  2010/07/05        2010/07/05      女         6   

     WORK_CITY WORK_PROVINCE WORK_COUNTRY   AGE       ...       ADD_Point_SUM  \
0            .            北京           CN  31.0       ...               39992   
1          NaN            北京           CN  42.0       ..

<a id="lrfmc_5_2"></a>
### 5.2 对数据进行统计

In [336]:
df_data_1_group = df_data_1.groupby('TYPE')
type_count = df_data_1_group.size().reset_index()
type_count.columns = ['TYPE','COUNT']
print(type_count)

     TYPE  COUNT
0      NA    944
1    一般客户  25360
2   低价值客户  12440
3  重要保持客户   5311
4  重要发展客户   2836
5  重要挽留客户  16097


In [340]:
%brunel data('type_count') bar x(TYPE) y(COUNT) color(COUNT) label(COUNT) style('.label{fill:white;text-shadow:none}') sort(TYPE) effect(enter:2000)  title("航空公司客户类型统计")  axes(y:2000:grid:"数量",x:6:"客户种类"):: width=800, height=300

<IPython.core.display.Javascript object>

In [341]:
#筛选出中国的数据
df_data_1_type_1 = df_data_1[(df_data_1.TYPE == '重要发展客户') & (df_data_1.WORK_COUNTRY == 'CN')]
df_data_1_type_1 = df_data_1_type_1.dropna()
#去除地址不正确的数据
del_array = ['*', '-', '.','.北京','BEIJING','BEIJING','BRIARC','Beijing','Beijing','HONGKONG','HRB','HRB','IRVINE','LEEWOLDE','N','N','OSAKA','PARIS','STANLEY','TOKYO','leon','rotteadam','stratford','。','。','。','。']
df_data_1_type_1_tmp = df_data_1_type_1[~ df_data_1_type_1.WORK_CITY.isin(del_array)]
df_data_1_type_1_tmp['COUNT'] = 1
print(df_data_1_type_1_tmp.head(5))

     MEMBER_NO    TYPE    FFP_DATE FIRST_FLIGHT_DATE GENDER  FFP_TIER  \
291      49453  重要发展客户  2010/02/26        2010/02/26      男         4   
318      62340  重要发展客户  2010/10/21        2010/11/24      男         6   
322      59920  重要发展客户  2011/09/18        2011/09/18      男         6   
348      44368  重要发展客户  2010/10/02        2010/10/19      男         5   
370      54913  重要发展客户  2006/09/18        2007/03/04      男         6   

    WORK_CITY WORK_PROVINCE WORK_COUNTRY   AGE  ...  Eli_Add_Point_Sum  \
291      克拉玛依            新疆           CN  62.0  ...               7321   
318       鄯善县            新疆           CN  47.0  ...               7020   
322        贵阳            贵州           CN  41.0  ...              15175   
348      乌鲁木齐            新疆           CN  51.0  ...              12369   
370        深圳            广东           CN  38.0  ...               5606   

     L1Y_ELi_Add_Points  Points_Sum  L1Y_Points_Sum  Ration_L1Y_Flight_Count  \
291                7321      108091 

/gpfs/fs01/user/sf31-bf990ab5b3efba-4276a445e7a2/.local/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [342]:
%brunel data('df_data_1_type_1_tmp') map('china') sum(COUNT) x(WORK_CITY) color(WORK_CITY) label(WORK_CITY) tooltip(WORK_CITY,#all) legends(auto) interaction(select:mouseover) title("高额交际费数据分析-地域") :: width=800,height=500

<IPython.core.display.Javascript object>